In [1]:
library(caret, quiet = TRUE)
library(base64enc)
library(httr, quiet = TRUE)

library(mlbench)


Attaching package: ‘httr’

The following object is masked from ‘package:caret’:

    progress



# Build a Model

In [2]:


data(BostonHousing)
BostonHousing$chas = as.numeric(BostonHousing$chas)

set.seed(1960)

dataset = BostonHousing[, -14] 

create_model  =  function() {

    model <- train(medv ~ ., data = BostonHousing, method = "rpart", preProcess=c("pca"))    

    return(model)
}


In [3]:
model = create_model()
# cat(model$feature_names)
# print(model)

Warning message in nominalTrainWorkflow(x = x, y = y, wts = weights, info = trainInfo, :
“There were missing values in resampled performance measures.”

In [4]:
pred_labels <- predict(model, BostonHousing[, -14] , type="raw")
df = data.frame(BostonHousing[,14])
names(df) = c("medv")
df$Estimator = pred_labels
df$Error = df$Estimator - df$medv
MAPE = mean(abs(df$Error / df$medv))
summary(df)
MAPE

      medv         Estimator         Error         
 Min.   : 5.00   Min.   :17.17   Min.   :-32.8329  
 1st Qu.:17.02   1st Qu.:17.17   1st Qu.: -2.5208  
 Median :21.20   Median :23.07   Median :  0.6734  
 Mean   :22.53   Mean   :22.53   Mean   :  0.0000  
 3rd Qu.:25.00   3rd Qu.:23.07   3rd Qu.:  3.7671  
 Max.   :50.00   Max.   :36.22   Max.   : 13.7157  

[1] 0.2358512

# SQL Code Generation

In [5]:

test_ws_sql_gen = function(mod) {
    WS_URL = "https://sklearn2sql.herokuapp.com/model"
    WS_URL = "http://localhost:1888/model"
    model_serialized <- serialize(mod, NULL)
    b64_data = base64encode(model_serialized)
    data = list(Name = "xgboost_test_model", SerializedModel = b64_data , SQLDialect = "postgresql" , Mode="caret")
    r = POST(WS_URL, body = data, encode = "json")
    # print(r)
    content = content(r)
    # print(content)
    lSQL = content$model$SQLGenrationResult[[1]]$SQL # content["model"]["SQLGenrationResult"][0]["SQL"]
    return(lSQL);
}

In [6]:
lModelSQL = test_ws_sql_gen(model)
cat(lModelSQL)


WITH "CenteredDataForPCA" AS 
(SELECT "ADS"."KEY" AS "KEY", (CAST("ADS"."Feature_0" AS FLOAT) - 3.6135235573122526) / 8.60154510533249 AS "Feature_0", (CAST("ADS"."Feature_1" AS FLOAT) - 11.363636363636365) / 23.322452994515135 AS "Feature_1", (CAST("ADS"."Feature_2" AS FLOAT) - 11.13677865612648) / 6.860352940897585 AS "Feature_2", (CAST("ADS"."Feature_3" AS FLOAT) - 1.0691699604743083) / 0.2539940413404104 AS "Feature_3", (CAST("ADS"."Feature_4" AS FLOAT) - 0.5546950592885376) / 0.115877675667556 AS "Feature_4", (CAST("ADS"."Feature_5" AS FLOAT) - 6.284634387351779) / 0.7026171434153233 AS "Feature_5", (CAST("ADS"."Feature_6" AS FLOAT) - 68.57490118577077) / 28.148861406903617 AS "Feature_6", (CAST("ADS"."Feature_7" AS FLOAT) - 3.7950426877470353) / 2.105710126627611 AS "Feature_7", (CAST("ADS"."Feature_8" AS FLOAT) - 9.549407114624506) / 8.707259384239368 AS "Feature_8", (CAST("ADS"."Feature_9" AS FLOAT) - 408.2371541501977) / 168.537116054959 AS "Feature_9", (CAST("ADS"."Feature_10

# Execute the SQL Code

In [7]:
library(RODBC)
conn = odbcConnect("pgsql", uid="db", pwd="db", case="nochange")
odbcSetAutoCommit(conn , autoCommit = TRUE)

[1] 0

In [8]:
df_sql = dataset
names(df_sql) = sprintf("Feature_%d",0:(ncol(df_sql)-1))
df_sql$KEY = seq.int(nrow(dataset))

sqlDrop(conn , "INPUT_DATA" , errors = FALSE)
sqlSave(conn, df_sql, tablename = "INPUT_DATA", verbose = FALSE)

# df_sql

In [9]:
colnames(df_sql)
# odbcGetInfo(conn)
# sqlTables(conn)

[1] "Feature_0"  "Feature_1"  "Feature_2"  "Feature_3"  "Feature_4" 
 [6] "Feature_5"  "Feature_6"  "Feature_7"  "Feature_8"  "Feature_9" 
[11] "Feature_10" "Feature_11" "Feature_12" "KEY"

In [10]:
df_sql_out = sqlQuery(conn, lModelSQL)
head(df_sql_out[order(df_sql_out$KEY),])

KEY,Estimator
1,23.07344
2,23.07344
3,23.07344
4,23.07344
5,23.07344
6,23.07344


In [11]:
# df_sql_out

# R RPART Output

In [12]:
estimator  =  predict(model, dataset, type = "raw")
df_r_out = data.frame(estimator)
names(df_r_out) = c("Estimator")

df_r_out$KEY = seq.int(nrow(dataset))
head(df_r_out)


Estimator,KEY
23.07344,1
23.07344,2
23.07344,3
23.07344,4
23.07344,5
23.07344,6


# Compare R and SQL output

In [13]:
df_merge = merge(x = df_r_out, y = df_sql_out, by = "KEY", all = TRUE, , suffixes = c("_1","_2"))
head(df_merge)

KEY,Estimator_1,Estimator_2
1,23.07344,23.07344
2,23.07344,23.07344
3,23.07344,23.07344
4,23.07344,23.07344
5,23.07344,23.07344
6,23.07344,23.07344


In [14]:
df_merge$Error = df_merge$Estimator_1 - df_merge$Estimator_2
df_merge$AbsError = abs(df_merge$Error)
head(df_merge)


KEY,Estimator_1,Estimator_2,Error,AbsError
1,23.07344,23.07344,3.552714e-15,3.552714e-15
2,23.07344,23.07344,3.552714e-15,3.552714e-15
3,23.07344,23.07344,3.552714e-15,3.552714e-15
4,23.07344,23.07344,3.552714e-15,3.552714e-15
5,23.07344,23.07344,3.552714e-15,3.552714e-15
6,23.07344,23.07344,3.552714e-15,3.552714e-15


In [15]:
df_merge_largest_errors = df_merge[df_merge$AbsError > 0.0001,]
df_merge_largest_errors

KEY,Estimator_1,Estimator_2,Error,AbsError


In [16]:
stopifnot(nrow(df_merge_largest_errors) == 0)

In [17]:
summary(df_sql_out)

      KEY          Estimator    
 Min.   :  1.0   Min.   :17.17  
 1st Qu.:127.2   1st Qu.:17.17  
 Median :253.5   Median :23.07  
 Mean   :253.5   Mean   :22.53  
 3rd Qu.:379.8   3rd Qu.:23.07  
 Max.   :506.0   Max.   :36.22  

In [18]:
summary(df_r_out)

   Estimator          KEY       
 Min.   :17.17   Min.   :  1.0  
 1st Qu.:17.17   1st Qu.:127.2  
 Median :23.07   Median :253.5  
 Mean   :22.53   Mean   :253.5  
 3rd Qu.:23.07   3rd Qu.:379.8  
 Max.   :36.22   Max.   :506.0  

In [19]:
summary(df_merge)

      KEY         Estimator_1     Estimator_2        Error           
 Min.   :  1.0   Min.   :17.17   Min.   :17.17   Min.   :-7.105e-15  
 1st Qu.:127.2   1st Qu.:17.17   1st Qu.:17.17   1st Qu.:-7.105e-15  
 Median :253.5   Median :23.07   Median :23.07   Median : 3.553e-15  
 Mean   :253.5   Mean   :22.53   Mean   :22.53   Mean   :-7.302e-16  
 3rd Qu.:379.8   3rd Qu.:23.07   3rd Qu.:23.07   3rd Qu.: 3.553e-15  
 Max.   :506.0   Max.   :36.22   Max.   :36.22   Max.   : 7.105e-15  
    AbsError        
 Min.   :3.553e-15  
 1st Qu.:3.553e-15  
 Median :7.105e-15  
 Mean   :5.757e-15  
 3rd Qu.:7.105e-15  
 Max.   :7.105e-15  